In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm, poisson
from datetime import datetime, timedelta
import random

In [21]:
# Date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

# Product categories
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Categories and their associated product names with realistic price ranges in Indian Rupees (₹)
categories = {
    'Beverages': [
        ('Coca-Cola', 40), ('Pepsi', 40), ('Sprite', 40),
        ('Orange Juice', 80), ('Apple Juice', 80), ('Milkshake', 120),
        ('Bottled Water', 20), ('Energy Drink', 100), ('Iced Coffee', 150),
        ('Green Tea', 60), ('Lemonade', 70)
    ],
    'Snacks': [
        ('Potato Chips', 20), ('Corn Chips', 20), ('Popcorn', 50),
        ('Chocolate Bar', 50), ('Cookies', 60), ('Crackers', 40),
        ('Granola Bar', 30), ('Pretzels', 40), ('Nuts Mix', 150),
        ('Candy Pack', 20), ('Cheese Puffs', 30)
    ],
    'Dairy': [
        ('Milk', 50), ('Cheese Block', 200), ('Yogurt', 60),
        ('Butter', 80), ('Sour Cream', 90), ('Cream Cheese', 150),
        ('Whipped Cream', 120), ('Ice Cream', 200), ('Eggs', 70),
        ('Almond Milk', 100), ('Greek Yogurt', 120)
    ],
    'Personal Care': [
        ('Shampoo', 150), ('Conditioner', 150), ('Body Wash', 120),
        ('Toothpaste', 80), ('Deodorant', 100), ('Face Wash', 200),
        ('Moisturizer', 300), ('Soap Bar', 30), ('Hair Gel', 120),
        ('Sunscreen', 400), ('Razors', 250)
    ],
    'Household': [
        ('Dish Soap', 50), ('Laundry Detergent', 300), ('All-Purpose Cleaner', 100),
        ('Paper Towels', 200), ('Trash Bags', 250), ('Sponges', 40),
        ('Air Freshener', 150), ('Bleach', 60), ('Fabric Softener', 200),
        ('Glass Cleaner', 80), ('Disinfectant Wipes', 150)
    ]
}

# Date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

# Random seed for reproducibility
np.random.seed(42)

# locations
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']


# Customer age groups
age_groups = ['18-25', '26-35', '36-45', '46-55', '56+']

# Gender options
genders = ['M', 'F', 'Other']

# Social sentiment range
sentiment_range = (-1, 1)

# Economic indicators (example ranges)
cpi_range = (100, 200)
unemployment_rate_range = (3, 10)
interest_rate_range = (0, 10)
gdp_growth_range = (-5, 5)

np.random.seed(42)

#### products.csv

In [22]:
# Generate products data
product_data = []
product_id_counter = 1

for category, products in categories.items():
    for product_name, base_price in products:
        # Add some variability to the price (±10%)
        price = round(base_price * np.random.uniform(0.9, 1.1), 2)
        
        # Random launch date within the date range
        launch_date = (start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days))).strftime('%Y-%m-%d')
        
        # Append product details
        product_data.append({
            'product_id': f'P{product_id_counter:05}',
            'category': category,
            'product_name': product_name,
            'price': price,  # Price in Indian Rupees
            'launch_date': launch_date
        })
        product_id_counter += 1

# Convert to DataFrame
products_df = pd.DataFrame(product_data)

# Save to CSV
products_df.to_csv('products.csv', index=False)

# Display the first few rows
print(products_df.head())

  product_id   category  product_name  price launch_date
0     P00001  Beverages     Coca-Cola  39.00  2022-05-10
1     P00002  Beverages         Pepsi  37.47  2022-12-31
2     P00003  Beverages        Sprite  40.79  2020-05-01
3     P00004  Beverages  Orange Juice  74.50  2020-11-26
4     P00005  Beverages   Apple Juice  79.35  2023-10-28


#### sales_data.csv

In [18]:
# Product categories
categories = ['Beverages', 'Snacks', 'Dairy', 'Personal Care', 'Household']

# locations
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']


# Date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

# Seasonality factors (monthly multipliers for each category)
seasonality_factors = {
    'Beverages': [1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3],  # Higher in summer
    'Snacks': [1.0, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.4, 1.2, 1.1, 1.0],     # Peak during holidays
    'Dairy': [1.0, 1.1, 1.2, 1.0, 0.9, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4, 1.5],       # Higher in winter
    'Personal Care': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],  # Consistent demand
    'Household': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]   # Consistent demand
}

# Base revenue growth rate (trend)
base_growth_rate = 0.001  # 0.1% daily growth

# Number of stores per region
num_stores_per_region = 5

# Price ranges for each category
price_ranges = {
    'Beverages': (1.0, 5.0),
    'Snacks': (0.5, 3.0),
    'Dairy': (2.0, 10.0),
    'Personal Care': (5.0, 20.0),
    'Household': (10.0, 50.0)
}
def generate_sales_data(num_products_per_category=10):
    sales_data = []
    sale_id_counter = 1
    current_date = start_date

    # Generate product IDs and prices
    products = []
    product_id_counter = 1
    for category in categories:
        for _ in range(num_products_per_category):
            price = round(np.random.uniform(*price_ranges[category]), 2)
            products.append({
                'product_id': f'P{product_id_counter:05}',
                'category': category,
                'price': price
            })
            product_id_counter += 1

    # Generate sales data
    while current_date <= end_date:
        for region in locations:
            for store_num in range(1, num_stores_per_region + 1):
                store_id = f'ST{region[0]}{store_num:02}'  # Example: SN01 for North Store 1
                for product in products:
                    # Seasonality and trend factors
                    month = current_date.month - 1
                    seasonality = seasonality_factors[product['category']][month]
                    trend = 1 + base_growth_rate * (current_date - start_date).days
                    base_units_sold = int(np.random.poisson(lam=50))  # Poisson distribution for units sold
                    units_sold = int(base_units_sold * seasonality * trend)

                    # Random discount (10% chance of promotion)
                    discount = round(np.random.uniform(0, 0.3), 2) if np.random.rand() < 0.1 else 0
                    revenue = units_sold * product['price'] * (1 - discount)

                    # Append record
                    sales_data.append({
                        'sale_id': f'S{sale_id_counter:07}',
                        'date': current_date.strftime('%Y-%m-%d'),
                        'product_id': product['product_id'],
                        'units_sold': units_sold,
                        'revenue': round(revenue, 2),
                        'store_id': store_id,
                        'region': region,
                        'discount_applied': discount
                    })
                    sale_id_counter += 1

        current_date += timedelta(days=1)

    return pd.DataFrame(sales_data)

# Generate sales data
sales_df = generate_sales_data()

# Save to CSV
sales_df.to_csv('fmcg_sales_data.csv', index=False)

# Display the first few rows
print(sales_df.head())


    sale_id        date product_id  units_sold  revenue store_id region  \
0  S0000001  2020-01-01     P00001          76   190.00    STN01  North   
1  S0000002  2020-01-01     P00002          74   355.20    STN01  North   
2  S0000003  2020-01-01     P00003          39   153.27    STN01  North   
3  S0000004  2020-01-01     P00004          54   183.06    STN01  North   
4  S0000005  2020-01-01     P00005          54    87.48    STN01  North   

   discount_applied  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


#### economic_indicators.csv

In [23]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Parameters
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

# Realistic ranges for economic indicators
cpi_range = (100, 300)  # Base CPI range
unemployment_rate_range = (3, 12)  # Percentage
interest_rate_range = (4, 8)  # Percentage
gdp_growth_range = (-5, 10)  # Percentage

# Correlation multipliers (to simulate relationships between indicators)
inflation_impact_on_interest_rate = 0.5  # Higher CPI increases interest rates
gdp_impact_on_unemployment = -0.2  # Higher GDP growth reduces unemployment

# Generate economic data
economic_data = []
current_date = start_date

while current_date <= end_date:
    for region in locations:
        # Simulate gradual trends over time
        months_since_start = (current_date.year - start_date.year) * 12 + (current_date.month - start_date.month)
        trend_factor = 1 + (months_since_start / 12) * 0.05  # 5% annual growth trend
        
        # Generate base economic indicators
        cpi = round(np.random.uniform(cpi_range[0], cpi_range[1]) * trend_factor, 2)
        gdp_growth = round(np.random.uniform(gdp_growth_range[0], gdp_growth_range[1]) * (1 + np.random.normal(0, 0.1)), 2)
        unemployment_rate = round(np.random.uniform(unemployment_rate_range[0], unemployment_rate_range[1]) + gdp_growth * gdp_impact_on_unemployment, 2)
        interest_rate = round(np.random.uniform(interest_rate_range[0], interest_rate_range[1]) + (cpi - cpi_range[0]) * inflation_impact_on_interest_rate / 100, 2)
        
        # Ensure no unrealistic values
        unemployment_rate = max(0, min(100, unemployment_rate))  # Unemployment between 0% and 100%
        gdp_growth = max(-10, min(20, gdp_growth))  # GDP growth between -10% and 20%
        
        # Append record
        economic_data.append({
            'date': current_date.strftime('%Y-%m-%d'),
            'region': region,
            'cpi': cpi,
            'unemployment_rate': unemployment_rate,
            'interest_rate': interest_rate,
            'gdp_growth': gdp_growth
        })
    
    # Move to the next month
    current_date += timedelta(days=30)

# Convert to DataFrame
economic_df = pd.DataFrame(economic_data)

# Save to CSV
economic_df.to_csv('economic_indicators.csv', index=False)

# Display the first few rows
print(economic_df.head())

         date       region     cpi  unemployment_rate  interest_rate  \
0  2020-01-01     New York  198.76               3.43           4.62   
1  2020-01-01  Los Angeles  227.28               7.63           8.27   
2  2020-01-01      Chicago  149.86               3.48           5.41   
3  2020-01-01      Houston  132.24               8.35           6.69   
4  2020-01-01      Phoenix  274.29               9.46           6.14   

   gdp_growth  
0        2.73  
1       -0.28  
2        1.06  
3        9.61  
4        8.02  


#### social_sentiment.csv

In [7]:
# Generate social_sentiment.csv
social_data = []
current_date = start_date

while current_date <= end_date:
    for category in categories:
        social_data.append({
            'date': current_date.strftime('%Y-%m-%d'),
            'category': category,
            'avg_sentiment': round(np.random.uniform(*sentiment_range), 2),
            'mentions': np.random.randint(100, 1000),
            'engagements': np.random.randint(500, 5000)
        })
    current_date += timedelta(days=7)  # Weekly data

social_df = pd.DataFrame(social_data)
social_df.to_csv('social_sentiment.csv', index=False)

#### customer_data.csv

In [24]:
# Generate customer_data.csv
customer_data = []
num_customers = 1000

for i in range(num_customers):
    signup_date = start_date + timedelta(days=np.random.randint(0, 1095))
    churned = np.random.rand() < 0.2  # 20% churn rate
    retention_score = round(norm.rvs(loc=0.7, scale=0.1), 2) if not churned else round(norm.rvs(loc=0.3, scale=0.1), 2)

    customer_data.append({
        'customer_id': f'C{i+1:05}',
        'signup_date': signup_date.strftime('%Y-%m-%d'),
        'region': random.choice(locations),
        'age_group': random.choice(age_groups),
        'gender': random.choice(genders),
        'retention_score': max(0, min(1, retention_score)),  # Clamp between 0 and 1
        'is_churned': churned
    })

customer_df = pd.DataFrame(customer_data)
customer_df.to_csv('customer_data.csv', index=False)

#### customer_purchases.csv

In [25]:
# Generate customer_purchases.csv
purchase_data = []
purchase_id_counter = 1

for _, customer in customer_df.iterrows():
    num_purchases = np.random.randint(1, 20)  # Random number of purchases per customer
    for _ in range(num_purchases):
        product = products_df.iloc[np.random.randint(0, len(products_df))]
        purchase_date = datetime.strptime(customer['signup_date'], '%Y-%m-%d') + timedelta(days=np.random.randint(0, 1000))
        quantity = np.random.randint(1, 5)
        total_price = round(quantity * product['price'] * (1 - np.random.uniform(0, 0.2)), 2)  # Apply random discount

        purchase_data.append({
            'purchase_id': f'PU{purchase_id_counter:07}',
            'customer_id': customer['customer_id'],
            'product_id': product['product_id'],
            'purchase_date': purchase_date.strftime('%Y-%m-%d'),
            'quantity': quantity,
            'total_price': total_price
        })
        purchase_id_counter += 1

purchase_df = pd.DataFrame(purchase_data)
purchase_df.to_csv('customer_purchases.csv', index=False)

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
from textblob import TextBlob

In [17]:
# Parameters
np.random.seed(42)  # For reproducibility
fake = Faker()

# Product categories
categories = ['Beverages', 'Snacks', 'Dairy', 'Personal Care', 'Household']

# Locations
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']

# Date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

# Review templates
positive_templates = [
    "I absolutely love this product! It's worth every penny.",
    "Great quality and fast delivery. Highly recommend!",
    "This is my go-to brand for {category}. Never disappoints.",
    "Perfect for daily use. Couldn't be happier with my purchase."
]

negative_templates = [
    "Very disappointed with this product. It broke after one use.",
    "The quality is terrible. Would not recommend.",
    "Not what I expected. Feels cheap and overpriced.",
    "Delivery was late, and the item was damaged."
]

neutral_templates = [
    "It's okay, nothing special.",
    "Does the job, but there are better options out there.",
    "Average product, neither good nor bad.",
    "Decent quality, but could improve."
]

In [14]:
def generate_reviews(num_reviews=1000):
    reviews = []
    review_id_counter = 1

    for _ in range(num_reviews):
        # Randomly select metadata
        product_id = f'P{random.randint(1, 100):05}'
        customer_id = f'C{random.randint(1, 500):05}'
        category = random.choice(categories)
        location = random.choice(locations)
        timestamp = fake.date_time_between(start_date=start_date, end_date=end_date)

        # Randomly assign a rating
        rating = random.randint(1, 5)

        # Select a review template based on rating
        if rating >= 4:
            template = random.choice(positive_templates)
            sentiment_score = 1  # Positive
        elif rating == 3:
            template = random.choice(neutral_templates)
            sentiment_score = 0  # Neutral
        else:
            template = random.choice(negative_templates)
            sentiment_score = -1  # Negative

        # Personalize the review text
        review_text = template.format(category=category)

        # Append review data
        reviews.append({
            'review_id': f'R{review_id_counter:07}',
            'product_id': product_id,
            'customer_id': customer_id,
            'category': category,
            'rating': rating,
            'review_text': review_text,
            'sentiment_score': sentiment_score,
            'timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            'location': location
        })

        review_id_counter += 1

    return pd.DataFrame(reviews)

In [15]:
# Generate 10,000 reviews
reviews_df = generate_reviews(num_reviews=10000)

# Save to CSV
reviews_df.to_csv('retail_reviews.csv', index=False)

# Display the first few rows
print(reviews_df.head())

  review_id product_id customer_id   category  rating  \
0  R0000001     P00098      C00002      Dairy       4   
1  R0000002     P00050      C00352  Household       1   
2  R0000003     P00095      C00191  Household       2   
3  R0000004     P00061      C00439     Snacks       5   
4  R0000005     P00075      C00043  Beverages       4   

                                         review_text  sentiment_score  \
0  Perfect for daily use. Couldn't be happier wit...                1   
1       Delivery was late, and the item was damaged.               -1   
2      The quality is terrible. Would not recommend.               -1   
3  Perfect for daily use. Couldn't be happier wit...                1   
4  I absolutely love this product! It's worth eve...                1   

             timestamp     location  
0  2023-04-17 06:41:48  Los Angeles  
1  2020-05-12 02:18:37     New York  
2  2021-10-19 18:39:58     New York  
3  2020-11-19 02:21:17     New York  
4  2022-06-23 12:12:02      